In [1]:
import torch
torch.__version__

'2.7.0'

In [2]:
torch.cuda.is_available()

False

In [3]:
print(torch.backends.mps.is_available())

True


In [6]:
tensor1d = torch.tensor([1, 2, 3])
print(tensor1d.dtype)

torch.int64


In [7]:
floatvec = torch.tensor([1.0, 2.0, 3.0])
print(floatvec.dtype)

torch.float32


In [8]:
floatvec = tensor1d.to(torch.float32)
print(floatvec.dtype)

torch.float32


In [9]:
tensor2d = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(tensor2d)

tensor([[1, 2, 3],
        [4, 5, 6]])


In [10]:
print(tensor2d.shape)

torch.Size([2, 3])


In [11]:
print(tensor2d.reshape(3, 2))

tensor([[1, 2],
        [3, 4],
        [5, 6]])


In [13]:
print(tensor2d.view(3,2))

tensor([[1, 2],
        [3, 4],
        [5, 6]])


In [14]:
print(tensor2d.T)

tensor([[1, 4],
        [2, 5],
        [3, 6]])


In [15]:
print(tensor2d.matmul(tensor2d.T))

tensor([[14, 32],
        [32, 77]])


In [17]:
print(tensor2d @ tensor2d.T)

tensor([[14, 32],
        [32, 77]])


In [20]:

import torch.nn.functional as F
from torch.autograd import grad

# ロジスティック回帰のフィードフォワードパス
y = torch.tensor([1.0]) # 正解ラベル
x1 = torch.tensor([1.1]) # 入力特徴量
w1 = torch.tensor([2.2], requires_grad=True) # 重みパラメータ
b = torch.tensor([0.0], requires_grad=True) # バイアスユニット
z = x1 * w1 + b # 線形変換
a = torch.sigmoid(z) # 活性化と出力
loss = F.binary_cross_entropy(a, y) 

# autogradによる勾配の計算
grad_L_w1 = grad(loss, w1, retain_graph=True) 
grad_L_b = grad(loss, b, retain_graph=True)

In [21]:
print(grad_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


In [22]:
loss.backward()  # 勾配を計算し、パラメータの.grad属性に格納
print(w1.grad)  # 重みの勾配
print(b.grad)   # バイアスの勾配

tensor([-0.0898])
tensor([-0.0817])


In [29]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),

            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            torch.nn.Linear(20, num_outputs)
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits

In [30]:
model = NeuralNetwork(num_inputs=50, num_outputs=3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [31]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total number of trainable parameters: {num_params}")

Total number of trainable parameters: 2213


In [32]:
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0535, -0.0516, -0.0774,  ...,  0.1327,  0.0212,  0.0655],
        [-0.0100,  0.0789, -0.0292,  ..., -0.1233, -0.1116,  0.0746],
        [ 0.0961, -0.0658, -0.0510,  ...,  0.0365,  0.0272,  0.1010],
        ...,
        [ 0.1343, -0.0493, -0.0259,  ...,  0.0031,  0.0310,  0.1026],
        [ 0.1090,  0.0455,  0.0162,  ..., -0.0205,  0.0483, -0.1103],
        [-0.0677,  0.0007,  0.0932,  ...,  0.0872, -0.0192,  0.1355]],
       requires_grad=True)


In [33]:
print(model.layers[0].weight.shape)

torch.Size([30, 50])


In [34]:
print(model.layers[0].bias)

Parameter containing:
tensor([ 0.0078,  0.1347, -0.0952,  0.0767,  0.1053, -0.0035, -0.0685,  0.0767,
         0.0021, -0.0352, -0.0358, -0.1231,  0.1151,  0.1071,  0.1031,  0.0454,
         0.0674,  0.0295, -0.0196, -0.0566,  0.0370,  0.0972,  0.1357, -0.0974,
         0.0106,  0.0944,  0.0559,  0.1049,  0.1274,  0.1345],
       requires_grad=True)


In [35]:
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=50, num_outputs=3)
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)


In [36]:
torch.manual_seed(123)
X = torch.rand((1,50))
out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]], grad_fn=<AddmmBackward0>)


In [37]:
with torch.no_grad():
    out = torch.softmax(model(X), dim=1)
print(out)

tensor([[0.3113, 0.3934, 0.2952]])


In [38]:
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5],
])

y_train = torch.tensor([0, 0, 0, 1, 1])  # クラスラベル

X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])
y_test = torch.tensor([0, 1])

In [42]:
from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y

    def __getitem__(self, index): # データレコードと対応するラベルを一つだけ取得
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y
    
    def __len__(self): # データセットのサイズを返す
        return self.labels.shape[0]

train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)        


In [61]:
from torch.utils.data import DataLoader

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True, 
    num_workers=0 
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False, # テストデータセットはシャッフルする必要はない
    num_workers=0
)

In [62]:
for i, (x, y) in enumerate(train_loader):
    print(f"Batch {i}:")
    print("Features:", x.shape)
    print("Labels:", y.shape)


Batch 0:
Features: torch.Size([2, 2])
Labels: torch.Size([2])
Batch 1:
Features: torch.Size([2, 2])
Labels: torch.Size([2])
Batch 2:
Features: torch.Size([1, 2])
Labels: torch.Size([1])


In [65]:
train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True  # 最後のバッチが小さい場合はドロップ)
)

In [66]:
for i, (x, y) in enumerate(train_loader):
    print(f"Batch {i}:")
    print("Features:", x.shape)
    print("Labels:", y.shape)

Batch 0:
Features: torch.Size([2, 2])
Labels: torch.Size([2])
Batch 1:
Features: torch.Size([2, 2])
Labels: torch.Size([2])


In [67]:
import torch.nn.functional as F

torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 3
for epoch in range(num_epochs):

    model.train()

    for batch_idx, (features, labels) in enumerate(train_loader):
        logits = model(features)

        loss = F.cross_entropy(logits, labels)

        optimizer.zero_grad() # 勾配を累積したくないため、前のイテレーションの勾配をゼロにする
        loss.backward() # 勾配を計算
        optimizer.step() # パラメータを更新

        print(f"Epoch {epoch + 1:03d}/{num_epochs:03d}, Batch: {batch_idx + 1:03d}/{len(train_loader):03d}, Loss: {loss:.2f}")




Epoch 001/003, Batch: 001/002, Loss: 0.75
Epoch 001/003, Batch: 002/002, Loss: 0.65
Epoch 002/003, Batch: 001/002, Loss: 0.44
Epoch 002/003, Batch: 002/002, Loss: 0.13
Epoch 003/003, Batch: 001/002, Loss: 0.03
Epoch 003/003, Batch: 002/002, Loss: 0.00


In [68]:
model.eval()
with torch.no_grad():
    outputs = model(X_train)
print(outputs)

tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])


In [69]:
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)

tensor([[    0.9991,     0.0009],
        [    0.9982,     0.0018],
        [    0.9949,     0.0051],
        [    0.0491,     0.9509],
        [    0.0307,     0.9693]])


In [70]:
predictions = torch.argmax(probas, dim=1)
print(predictions)

tensor([0, 0, 0, 1, 1])


In [72]:
predictions == y_train

tensor([True, True, True, True, True])

In [78]:
def compute_accuracy(model, dataloader):
    model = model.eval()
    correct = 0.0
    total_examples = 0.0

    for idx, (features, labels) in enumerate(dataloader):
        with torch.no_grad():
            logits = model(features)
        
        predictions = torch.argmax(logits, dim=1)
        compare = labels == predictions
        correct += torch.sum(compare)
        total_examples += len(compare)

    return (correct / total_examples).item() # item

In [79]:
print(compute_accuracy(model, train_loader))

1.0
